## Variable Selection

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
from math import sqrt

df = pd.read_csv('C:/Users/dstoc/Documents/Python Scripts/Fixed Income Dashboards/reduced_data_cleaned.csv')
df2 = df.copy()

In [ ]:
# Assuming 'Date' is the name of the column containing date information
df2['Date'] = pd.to_datetime(df2['Date'])
df2.set_index('Date', inplace=True)
df2 = df2.replace([np.inf, -np.inf], np.nan).dropna(axis = 1)

# Define the features (independent variables) and target (dependent variable)
features = list(df2)

In [ ]:
# Define the lag order (number of lags to create)
lag_order = 6  # Example: Creating lagged values for the past n periods
# Define the initial training size (e.g., 70% of the data)
initial_train_size = int(0.7 * len(df2)) - lag_order
InSamp_length = len(df2) - initial_train_size - lag_order

# Define the number of periods to lead the target variable (1 period in this case)
lead_periods = 1

In [ ]:
# redefine the features (independent variables) and target (dependent variable)
features = list(df2)

# Create a new variable representing the 'lead' of the target
df2['YIELD_weekly_percent_change_10 YR_lead'] = df2['YIELD_weekly_percent_change_10 YR'].shift(-lead_periods)

# Drop rows with missing values in the features or target (created by the shift)
df2 = df2.dropna(subset=features + ['YIELD_weekly_percent_change_10 YR_lead'])
df3 = df2.copy()
features = list(df3.columns)[0:(len(df3.columns)-1)]
X = df3[features]
y = df3['YIELD_weekly_percent_change_10 YR_lead']

target_var = 'YIELD_weekly_percent_change_10 YR_lead'

if target_var.lower() in map(str.lower, X.columns):
    print(f"The DataFrame contains the column '{target_var}'.")
else:
    print(f"The DataFrame X does not contain the target variable: '{target_var}'.")


Spearman's rank correlation is often used when the variables being compared may not have a linear relationship or when the assumptions of parametric correlation measures like Pearson's correlation are not met.

In [ ]:
df3.iloc[:InSamp_length,].tail()
import pandas as pd

def select_top_variables(df, target_variable, num_variables):
    # Calculate the rank correlation (Spearman's rho) with the target variable
    correlation_matrix = df.corrwith(df[target_variable], method='spearman')
    
    # Sort variables based on absolute correlation values
    sorted_variables = correlation_matrix.abs().sort_values(ascending=False)
    
    # Select the top variables
    selected_variables = sorted_variables.index[:num_variables]
    
    # Extract the correlation values for the selected variables
    selected_correlations = correlation_matrix[selected_variables]
    
    # Create a DataFrame to store the results
    corr_data = pd.DataFrame({
        'Variable': selected_variables,
        'Correlation': selected_correlations
    })
    
    return corr_data

# Assuming df3 is your DataFrame and 'YIELD_weekly_percent_change_10 YR' is the target variable
correlated_variables = select_top_variables(df3.loc[df3.index[:InSamp_length]], 'YIELD_weekly_percent_change_10 YR_lead', 200)

df3.loc[df3.index[:InSamp_length], correlated_variables['Variable']].head()

Variance Inflation Factor (VIF) is a tool used in multiple regression analysis to measure the degree of multicollinearity. Multicollinearity occurs when there is a correlation between multiple independent variables in a multiple regression model, which can adversely affect the regression results. The VIF estimates how much the variance of a regression coefficient is inflated due to multicollinearity. The dependent variable is the outcome being acted upon by the independent variables.

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

def calculate_vif(X):
    vif_data = pd.DataFrame()
    vif_data["Variable"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    return vif_data

# Assuming X is your DataFrame containing the selected variables
#vif_results = calculate_vif(df3.iloc[:InSamp_length,[correlated_variables['Variable']]])
vif_results = calculate_vif(df3.loc[df3.index[:InSamp_length], correlated_variables['Variable']])

# Print the variables with high VIF
low_vif_variables = vif_results.sort_values('VIF')[:51]
print("Variables with low VIF:")
print(low_vif_variables)


Save a pkl object that contains everything needed for the backtest

In [ ]:
selected_variables = low_vif_variables.copy()
class selected_variables_object:
  def __init__(self,selected_variables, target_var, y, best_estim_preds, final_model):
    self.selected_variables = selected_variables
    self.target_var = target_var
    self.y = y
    self.best_estim_preds = best_estim_preds
    self.final_model = best_gbm

backtest_object_10Y_UST_v1_1 = backtest_object(grid_search, X, y, best_estim_preds, best_gbm)
print(backtest_object_10Y_UST_v1_1)

import pickle

# Serialize the object using pickle and save to a file
with open('backtest_object_10Y_UST_v1_1.pkl', 'wb') as file:
    pickle.dump(backtest_object_10Y_UST_v1_1, file)